### 대여소 별 거리(도로 기반) 비용 생성
대여소 별 도로기반 거리비용을 생성하여, TSP cost로 사용

### bigquery 연동

In [1]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'AppleGothic'

In [2]:
#bigquery 연동하기 
from google.oauth2 import service_account
from google.cloud import bigquery

In [3]:
credentials = service_account.Credentials.from_service_account_file('./multi-final-project-65862f4bddb1.json')

In [4]:
credentials

In [5]:
client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id
)

In [6]:
client

In [7]:
pd.set_option('display.max_rows', None)  # 모든 행 표시
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.expand_frame_repr', False)  # 열이 화면 너비를 초과할 때 데이터프레임을 줄 바꿈하지 않도록 설정

In [8]:
# 텍스트 파일에서 대여소 리스트 불러오기
with open('/Users/nuri.park/Desktop/work_place/Seoul_bike_logistic_network/flask_server/station_id_list.txt', 'r') as file:
    station_id_list = [line.strip() for line in file.readlines()]

### 타겟 대여소 리스트 준비

In [9]:
pip install geopy


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [72]:
# 강남구 재고0인 상태 대여소
query = """
SELECT 
SUM(stock_0_count) AS no_stock,
NS.Rental_location_ID,
SI.Latitude,
SI.Longitude
FROM `multi-final-project.Preprocessing.1109_관리권역_재고0인시간대카운트` AS NS

JOIN
  `multi-final-project.Rental_Location_ID_master.Rental_Station_ID_master` AS SI
  ON
    NS.Rental_location_ID = SI.Rental_location_ID

WHERE
  stock_0_count = 1

GROUP BY
  NS.Rental_location_ID, SI.Rental_location_number,SI.Latitude, SI.Longitude

ORDER BY
  no_stock DESC
  """

In [73]:
no_stock_stations = client.query(query).to_dataframe()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [74]:
# 대여소 리스트와 일치하는 데이터만 필터링
same_stations = no_stock_stations[no_stock_stations['Rental_location_ID'].isin(station_id_list)]

In [75]:
# 상위 20% 재고부족 대여소
stations_32 = same_stations.head(32)

In [76]:
stations_32.reset_index(drop=True, inplace=True)

In [77]:
stations_32=stations_32[['Rental_location_ID','Latitude','Longitude']]

In [78]:
# 관리권역1
area_1 = {
    1: ['ST-1883', 'ST-2882', 'ST-1366', 'ST-1568','ST-1246', 'ST-3108', 
        'ST-789', 'ST-3208','ST-1108','ST-962', 'ST-961','ST-963',
        'ST-786', 'ST-2682','ST-953','ST-3164', 'ST-784']
}

In [79]:
# 딕셔너리의 모든 대여소 ID 추출
area_1 = [station for sublist in area_1.values() for station in sublist]

In [80]:

# 필터링된 데이터프레임 생성
area_1_df = stations_32[stations_32['Rental_location_ID'].isin(area_1)]

In [81]:
area_1_df.reset_index(drop=True,inplace=True)

In [84]:
# 관리권역2
area_2 = {
    2: ['ST-1561', 'ST-1577', 'ST-1562', 'ST-3115','ST-2684', 'ST-811', 
        'ST-809', 'ST-814','ST-1896','ST-959', 'ST-966','ST-812',
        'ST-960', 'ST-817','ST-816']
}

In [85]:
# 딕셔너리의 모든 대여소 ID 추출
area_2 = [station for sublist in area_2.values() for station in sublist]

In [86]:
area_2_df = stations_32[stations_32['Rental_location_ID'].isin(area_2)]

### TMAP API 호출 및 단일 거리계산

In [96]:
import requests
import time
import pandas as pd

# Tmap API 설정
API_KEY = "TMAP_API_KEY"  # 자신의 Tmap API 키를 입력하세요
url = "https://apis.openapi.sk.com/tmap/routes?version=1&format=json"

In [324]:
# 출발지와 도착지 정보
start_station = {"Rental_location_ID": 'center', 'Latitude':37.4957886, 'Longitude':127.0717955}

end_stations = [
    {"Rental_location_ID": 'ST-1561', "Latitude": 37.475986, "Longitude": 127.059624},
    {"Rental_location_ID": 'ST-1577', "Latitude": 37.498470, "Longitude": 127.030113},
    {"Rental_location_ID": 'ST-1896', "Latitude": 37.500244, "Longitude": 127.036057},
    {"Rental_location_ID": 'ST-3115', "Latitude": 37.497753, "Longitude": 127.038574},
    {"Rental_location_ID": 'ST-809', "Latitude": 37.502396, "Longitude": 127.058800},
    {"Rental_location_ID": 'ST-2684', "Latitude": 37.499866, "Longitude": 127.060501},
    {"Rental_location_ID": 'ST-816', "Latitude": 37.485157, "Longitude": 127.034073},
    {"Rental_location_ID": 'ST-817', "Latitude": 37.493759, "Longitude": 127.045898},
    {"Rental_location_ID": 'ST-966', "Latitude": 37.501663, "Longitude": 127.042976},
    {"Rental_location_ID": 'ST-812', "Latitude": 37.502357, "Longitude": 127.043190},
    {"Rental_location_ID": 'ST-959', "Latitude": 37.501545, "Longitude": 127.038567},
    {"Rental_location_ID": 'ST-814', "Latitude": 37.498051, "Longitude": 127.044601},
    {"Rental_location_ID": 'ST-1562', "Latitude": 37.474472, "Longitude": 127.055450},
    {"Rental_location_ID": 'ST-811', "Latitude": 37.503212, "Longitude": 127.042732},
    {"Rental_location_ID": 'ST-960', "Latitude": 37.494823, "Longitude": 127.047905}
]
    

In [325]:
# Tmap API 호출 함수
def get_tmap_distance(start_lat, start_lng, end_lat, end_lng, api_key):
    payload = {
        "startX": start_lng,
        "startY": start_lat,
        "endX": end_lng,
        "endY": end_lat,
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "searchOption": "0",  # 자동차 경로 옵션
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "appKey": api_key
    }
    try:
        response = requests.post(url, json=payload, headers=headers, timeout=10)
        if response.status_code == 200:
            result = response.json()
            return result["features"][0]["properties"]["totalDistance"]
        else:
            print(f"Error: {response.status_code}, {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

In [326]:
for end_station in end_stations:
    distance = get_tmap_distance(
        start_station["Latitude"], start_station["Longitude"],
        end_station["Latitude"], end_station["Longitude"],
        API_KEY
    )
    if distance is not None:
        data = {
            "Start_Station": start_station["Rental_location_ID"],
            "End_Station": end_station["Rental_location_ID"],
            "Distance_km": distance / 1000  # km로 변환
        }
        df_result = pd.DataFrame([data])
        print("결과 데이터프레임:")
        print(df_result)
    else:
        print(f"거리 계산 실패: {start_station['Rental_location_ID']} -> {end_station['Rental_location_ID']}")
    time.sleep(1)  # API 호출 제한을 위한 대기

결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center     ST-1561       11.496
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center     ST-1577        7.319
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center     ST-1896        7.072
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center     ST-3115         6.32
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center      ST-809        2.284
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center     ST-2684        3.347
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center      ST-816        5.613
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center      ST-817        5.449
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center      ST-966        4.065
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center      ST-812        5.723
결과 데이터프레임:
  Start_Station End_Station  Distance_km
0        center   

In [251]:
df_stations = area_2_df

### TMAP API 호출 및 다중 거리계산

In [252]:
# Tmap API 호출 함수
def get_tmap_distance(start_lat, start_lng, end_lat, end_lng, api_key):
    payload = {
        "startX": start_lng,
        "startY": start_lat,
        "endX": end_lng,
        "endY": end_lat,
        "reqCoordType": "WGS84GEO",
        "resCoordType": "WGS84GEO",
        "searchOption": "0",  # 자동차 경로 옵션
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "appKey": api_key
    }
    try:
        response = requests.post(url, json=payload, headers=headers, timeout=10)
        if response.status_code == 200:
            result = response.json()
            return result["features"][0]["properties"]["totalDistance"]
        else:
            print(f"Error: {response.status_code}, {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

In [253]:
# 다중 대여소 간 거리 계산
n_stations = len(df_stations)
distance_matrix = [[0 for _ in range(n_stations)] for _ in range(n_stations)]

for i in range(n_stations):
    for j in range(n_stations):
        if i != j:  # 동일 대여소는 거리 0
            start_lat = df_stations.iloc[i]["Latitude"]
            start_lng = df_stations.iloc[i]["Longitude"]
            end_lat = df_stations.iloc[j]["Latitude"]
            end_lng = df_stations.iloc[j]["Longitude"]
            print(f"Calculating: {df_stations.iloc[i]['Rental_location_ID']} → {df_stations.iloc[j]['Rental_location_ID']}")
            # API 호출
            distance = get_tmap_distance(start_lat, start_lng, end_lat, end_lng, API_KEY)
            if distance is not None:
                distance_matrix[i][j] = distance / 1000  # km로 변환
            else:
                distance_matrix[i][j] = None  # 에러 발생 시 None
            time.sleep(3)  # 요청 간 딜레이 추가 (3초)
        else:
            distance_matrix[i][j] = 0  # 동일 대여소는 거리 0

Calculating: ST-1561 → ST-1577
Calculating: ST-1561 → ST-1896
Calculating: ST-1561 → ST-3115
Calculating: ST-1561 → ST-809
Calculating: ST-1561 → ST-2684
Calculating: ST-1561 → ST-816
Calculating: ST-1561 → ST-817
Calculating: ST-1561 → ST-966
Calculating: ST-1561 → ST-812
Calculating: ST-1561 → ST-959
Calculating: ST-1561 → ST-814
Calculating: ST-1561 → ST-1562
Calculating: ST-1561 → ST-811
Calculating: ST-1561 → ST-960
Calculating: ST-1577 → ST-1561
Calculating: ST-1577 → ST-1896
Calculating: ST-1577 → ST-3115
Calculating: ST-1577 → ST-809
Calculating: ST-1577 → ST-2684
Calculating: ST-1577 → ST-816
Calculating: ST-1577 → ST-817
Calculating: ST-1577 → ST-966
Calculating: ST-1577 → ST-812
Calculating: ST-1577 → ST-959
Calculating: ST-1577 → ST-814
Calculating: ST-1577 → ST-1562
Calculating: ST-1577 → ST-811
Calculating: ST-1577 → ST-960
Calculating: ST-1896 → ST-1561
Calculating: ST-1896 → ST-1577
Calculating: ST-1896 → ST-3115
Calculating: ST-1896 → ST-809
Calculating: ST-1896 → ST-2

In [254]:
# 거리 행렬을 데이터프레임으로 변환
distance_df2 = pd.DataFrame(distance_matrix, 
                           columns=df_stations["Rental_location_ID"], 
                           index=df_stations["Rental_location_ID"])

In [255]:
distance_df2

Rental_location_ID,ST-1561,ST-1577,ST-1896,ST-3115,ST-809,ST-2684,ST-816,ST-817,ST-966,ST-812,ST-959,ST-814,ST-1562,ST-811,ST-960
Rental_location_ID,,,,,,,,,,,,,,,
ST-1561,0.000,5.956,6.341,4.625,3.727,3.305,4.250,4.122,4.474,4.396,5.364,3.953,2.198,4.496,3.264
ST-1577,10.995,0.000,0.563,1.386,3.081,3.641,2.327,2.105,1.296,1.720,1.460,1.720,4.711,1.276,2.351
ST-1896,10.445,1.085,0.000,0.824,2.492,3.079,2.519,1.514,0.733,1.157,0.898,1.157,4.160,0.713,1.788
ST-3115,9.952,1.840,0.752,0.000,2.763,3.350,2.067,1.092,1.005,1.429,1.169,1.429,3.697,1.510,1.653
ST-809,10.626,3.583,2.923,2.171,0.000,0.514,4.007,2.576,1.909,1.831,2.500,1.444,4.341,1.931,1.781
ST-2684,10.481,3.922,3.360,2.509,0.422,0.000,3.862,3.688,2.248,2.170,2.570,1.782,4.196,2.219,2.119
ST-816,10.765,1.706,2.311,2.033,3.858,3.437,0.000,1.874,3.000,2.922,3.202,2.478,3.504,3.022,2.436
ST-817,8.890,2.604,2.576,1.192,1.984,1.563,2.245,0.000,1.126,1.048,2.016,0.604,2.606,1.147,0.561
ST-966,9.736,2.588,1.174,1.175,1.800,2.387,2.815,1.577,0.000,0.802,1.305,0.448,3.451,0.902,1.079


In [256]:
distance_df2.to_csv("도로기반_대여소별_거리_관리권역2.csv")